<a href="https://colab.research.google.com/github/Vicky270506/GenAI/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import gradio as gr

# Download required NLTK resources
nltk.download('vader_lexicon')

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    # Get sentiment scores
    scores = sia.polarity_scores(text)

    # Determine sentiment label
    compound_score = scores['compound']
    if compound_score >= 0.05:
        sentiment = "Positive 😊"
    elif compound_score <= -0.05:
        sentiment = "Negative 😠"
    else:
        sentiment = "Neutral 😐"

    # Format results
    result = f"""
    Sentiment: {sentiment}
    Positive: {scores['pos']:.2f}
    Neutral: {scores['neu']:.2f}
    Negative: {scores['neg']:.2f}
    Compound: {scores['compound']:.2f}
    """
    return result

# Create Gradio interface
demo = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter text for sentiment analysis..."),
    outputs="text",
    title="📊 Sentiment Analyzer",
    description="Classify text sentiment using NLTK's VADER analyzer",
    examples=[
        ["I absolutely love this product! Best purchase ever!"],
        ["This service is terrible and disappointing."],
        ["The weather is neither good nor bad today."]
    ]
)

# Launch the application
if __name__ == "__main__":
    demo.launch()


In [ ]:
import spacy
import gradio as gr

# Load the English language model from spaCy
nlp = spacy.load("en_core_web_sm")

def pos_tagging(text):
    # Process the input text using spaCy
    doc = nlp(text)

    # Create a list to hold the results
    results = []

    # Extract tokens and their POS tags
    for token in doc:
        results.append(f"{token.text} | {token.pos_} | {spacy.explain(token.pos_)}")

    # Join results into a single string for display
    return "\n".join(results)

# Create Gradio interface
demo = gr.Interface(
    fn=pos_tagging,
    inputs=gr.Textbox(lines=5, placeholder="Enter text for POS tagging..."),
    outputs="text",
    title="📝 Part-Of-Speech Tagging",
    description="Enter a sentence to see the part-of-speech tags for each word.",
    examples=[
        ["Apple is planning to buy an Indian startup for $1 billion."],
        ["The quick brown fox jumps over the lazy dog."],
        ["I love programming in Python!"]
    ]
)

# Launch the application
if __name__ == "__main__":
    demo.launch()


In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import gradio as gr

# Create a new chatbot instance
chatbot = ChatBot(
    "MeeshoBot",
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        'chatterbot.logic.BestMatch'
    ]
)

# Training data for the chatbot
conversation = [
    "Hi",
    "Hello! How can I assist you today?",

    "What is the status of my order?",
    "Please provide your order ID to check the status.",

    "When will my delivery arrive?",
    "Deliveries usually arrive within 3-5 business days. Please check your order details for specific dates.",

    "I have an issue with my payment.",
    "I'm sorry to hear that. Can you please provide more details about the payment issue?",

    "Can I change my order?",
    "You can change your order within 30 minutes of placing it. Please provide your order ID.",

    "Thank you",
    "You're welcome! If you have any more questions, feel free to ask."
]

# Train the chatbot
trainer = ListTrainer(chatbot)
trainer.train(conversation)

def get_response(user_input):
    # Get response from the chatbot
    return str(chatbot.get_response(user_input))

# Gradio interface function
def chatbot_interface(user_input, language, gender):
    response = get_response(user_input)

    # Customize response based on language and gender preferences
    if language == 'Hindi':
        response = f"यहाँ आपकी मदद के लिए है: {response}"

    if gender == 'Female':
        response += " 😊 (female voice)"

    return response

# Create Gradio interface
demo = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Textbox(lines=2, placeholder="Ask me anything about orders or payments..."),
        gr.Radio(["English", "Hindi"], label="Select Language"),
        gr.Radio(["Male", "Female"], label="Select Gender")
    ],
    outputs="text",
    title="Meesho Customer Support Chatbot",
    description="A chatbot to assist with queries regarding orders, deliveries, and payment issues."
)

# Launch the application
if __name__ == "__main__":
    demo.launch()


In [ ]:
import json
import random
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from gtts import gTTS
import pickle
import gradio as gr
nltk.download('punkt_tab')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents.json file (sample intents for Meesho chatbot)
intents = {
    "intents": [
        {"tag": "greeting", "patterns": ["Hi", "Hello", "Hey"], "responses": ["Hello! How can I assist you today?"]},
        {"tag": "order_status", "patterns": ["What is the status of my order?", "Track my order"], "responses": ["Please provide your order ID to check the status."]},
        {"tag": "delivery", "patterns": ["When will my delivery arrive?", "Delivery details"], "responses": ["Deliveries usually arrive within 3-5 business days."]},
        {"tag": "payment_issue", "patterns": ["I have a payment issue", "Payment failed"], "responses": ["I'm sorry to hear that. Please provide more details about the payment issue."]},
        {"tag": "goodbye", "patterns": ["Bye", "See you later"], "responses": ["Goodbye! Have a great day!"]}
    ]
}

# Preprocessing data
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]))
classes = sorted(set(classes))

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1 if word in word_patterns else 0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Convert training data to NumPy arrays with consistent dimensions
train_x = np.array([row[0] for row in training])
train_y = np.array([row[1] for row in training])

# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Save the model and data structures for future use
model.save("meesho_chatbot_model.h5")
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

# Load trained model and data structures
model = tf.keras.models.load_model("meesho_chatbot_model.h5")
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow_data = bow(sentence, words)
    res = model.predict(np.array([bow_data]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

def get_response(intents_list):
    tag = intents_list[0]['intent']
    for intent in intents['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])

def chatbot_response(user_input):
    intents_list = predict_class(user_input)
    response_text = get_response(intents_list)
    return response_text

def generate_audio(text, language="en", gender="male"):
    tts_voice_option = gTTS(text=text, lang=language)
    audio_file_path = f"response_{gender}_{language}.mp3"
    tts_voice_option.save(audio_file_path)
    return audio_file_path

def chatbot_interface(user_input, language="en", gender="male"):
    response_text = chatbot_response(user_input)

    # Generate audio response based on language and gender preferences
    audio_path = generate_audio(response_text, language=language, gender=gender)

    return response_text, audio_path

# Gradio Interface
demo_ui = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Textbox(lines=2, placeholder="Ask me anything about orders or payments..."),
        gr.Radio(["en", "hi"], label="Select Language"),
        gr.Radio(["male", "female"], label="Select Gender")
    ],
    outputs=["text", gr.Audio()],
    title="Meesho Customer Support Chatbot",
    description="A chatbot to assist with queries regarding orders, deliveries, and payment issues."
)

if __name__ == "__main__":
    demo_ui.launch()


In [ ]:
import requests
from bs4 import BeautifulSoup
import gradio as gr

def get_song_data(query):
    # Search for songs related to the query
    search_url = f"https://search.azlyrics.com/search.php?q={query.replace(' ', '+')}"
    try:
        response = requests.get(search_url, timeout=10)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.text, "lxml")
    except requests.exceptions.RequestException as e:
        return f"Error: Could not retrieve search results. {e}", [], []

    # Extract song links and titles
    matches = soup.find_all('td', class_='text-left visitedlyr')

    if not matches:
        return "No songs found.", [], []

    songs = []
    for match in matches:
        try:
            a_tag = match.find('a')
            if a_tag:
                title = a_tag.text
                link = a_tag['href']
                songs.append((title, link))
            else:
                print("Warning: No <a> tag found in the expected <td>.")
                continue
        except Exception as e:
            print(f"Error extracting song: {e}")
            continue

    if not songs:
        return "No songs found.", [], []

    # Get the most popular song (first in the list)
    most_popular_song = songs[0]

    # Fetch lyrics for the most popular song
    lyrics_url = most_popular_song[1]
    try:
        lyrics_response = requests.get(lyrics_url, timeout=10)
        lyrics_response.raise_for_status()
        lyrics_soup = BeautifulSoup(lyrics_response.text, "lxml")
        lyrics_div = lyrics_soup.find('div', class_='col-xs-12 col-lg-8 text-center')
        if lyrics_div:
            lyrics = lyrics_div.get_text(separator="\n").strip()
        else:
            lyrics = "Lyrics not found."
    except requests.exceptions.RequestException as e:
        lyrics = f"Error: Could not retrieve lyrics. {e}"
    except Exception as e:
        lyrics = f"Error processing lyrics: {e}"

    # Return the most popular song, its lyrics, and other related songs
    related_songs_list = [song[0] for song in songs[1:]]
    return most_popular_song[0], lyrics, related_songs_list

def song_finder(input_text):
    title, lyrics, related_songs = get_song_data(input_text)

    return title, lyrics, related_songs

# Create Gradio interface
demo = gr.Interface(
    fn=song_finder,
    inputs=gr.Textbox(lines=2, placeholder="Enter a phrase or keyword..."),
    outputs=[
        gr.Textbox(label="Most Popular Song"),
        gr.Textbox(label="Lyrics", lines=10),
        gr.Dataframe(label="Related Songs", headers=["Song Titles"], row_count="dynamic")
    ],
    title="Song Finder",
    description="Enter a phrase to find related songs and their lyrics."
)

if __name__ == "__main__":
    demo.launch()


In [ ]:
import lyricsgenius
import gradio as gr
import json

# Replace with your Genius API access token
GENIUS_ACCESS_TOKEN = "EBb6_tnc9cH7ERPIfx7FC3lT1Tl5M9CDscsUrS6N_9bFDAwRhXG5iLteesTOY1Gh"

genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)
genius.verbose = False  # Suppress verbose output
genius.remove_section_headers = True  # Remove section headers from lyrics

def get_song_data(query):
    try:
        # Search for the song
        songs = genius.search(query, per_page=5)  # Request multiple results

        if songs and songs['hits']:  # Ensure results are not empty
            first_song = songs['hits'][0]['result']  # Access the first result

            song = genius.song(first_song['id'])  # Get song details by ID

            if song:
                title = song.title
                lyrics = song.lyrics
                artist = song.artist

                return f"{title} by {artist}", lyrics, []
            else:
                return "Song details not found.", "", []
        else:
            return "No songs found.", "", []

    except Exception as e:
        return f"Error: {e}", "", []

def song_finder(input_text):
    title, lyrics, related_songs = get_song_data(input_text)

    return title, lyrics, related_songs

# Create Gradio interface
demo = gr.Interface(
    fn=song_finder,
    inputs=gr.Textbox(lines=2, placeholder="Enter a phrase or keyword..."),
    outputs=[
        gr.Textbox(label="Most Popular Song"),
        gr.Textbox(label="Lyrics", lines=10),
        gr.Dataframe(label="Related Songs", headers=["Song Titles"], row_count="dynamic")
    ],
    title="Song Finder",
    description="Enter a phrase to find related songs and their lyrics."
)

if __name__ == "__main__":
    demo.launch()
